In [14]:
#import necessory libraries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import tqdm

In [15]:
# Set up the Chrome WebDriver
driver = webdriver.Chrome()

In [16]:
# Amazon URL for product search
URL = "https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar"
driver.get(URL)

In [17]:
#maximize the window
driver.maximize_window()

In [18]:
# Lists to store scraped data
Product_Name_List = []
Price_List = []
Rating_List = []

In [19]:
# Find all product elements on the page
products = driver.find_elements(By.XPATH, "//span[@class='rush-component s-latency-cf-section']//div[@data-component-type='s-search-result']")

In [12]:

# Loop through each product and scrape information
for product in tqdm.tqdm(products):
    # Extract product name
    product_name = product.find_element(By.XPATH, ".//h2/a/span").text
    
    # Wait for the price element to be present
    WebDriverWait(product, 10).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-price']")))
    
    # Extract product price
    try:
        product_price_element = product.find_element(By.XPATH, ".//span[@class='a-price']")
        # Use encode and decode to handle special characters
        product_price = product_price_element.text.encode('utf-8').decode('utf-8')
    except NoSuchElementException:
        product_price = "Price not available"
    
    # Wait for the rating element to be present
    WebDriverWait(product, 10).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-icon-alt']")))
    
    # Try to extract product rating, handle the case where it's not available
    try:
        product_rating = product.find_element(By.XPATH, ".//span[@class='a-icon-alt']").get_attribute("innerHTML")
    except NoSuchElementException:
        product_rating = "Rating not available"

    # Print the scraped information for the current product
    print(product_name, "-----", product_price, "-----", product_rating)

    # Append information to lists for later use
    Product_Name_List.append(product_name)
    Price_List.append(product_price)
    Rating_List.append(product_rating)

  0%|                                                                                           | 0/33 [00:16<?, ?it/s]


MaxRetryError: HTTPConnectionPool(host='localhost', port=4406): Max retries exceeded with url: /session/859b7d4c0e354de4652031e063b1fa6b/element/4AF4328767AA2FCC462B71B608E403E8_element_130/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000180267E7F50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [9]:
# Create a DataFrame using the scraped data
df_products = pd.DataFrame({
    'Product_Name': Product_Name_List,
    'Product_Price': Price_List,
    'Product_Ratings': Rating_List
})

# Save the DataFrame to a CSV file named 'Product_details.csv'
df_products.to_csv('Product_details.csv', index=False)

In [20]:
# Close the Chrome WebDriver
driver.quit()